#E1_VF_VD

#1. Mount Google Drive and install libraries if needed

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
try:
    import torch
    import torchvision
    import pytorch3d
    import trimesh
    import tensorflow_graphics
    print("Libraries successfully imported")
except Exception as e:
    print("Modules not found, but will now install")
    !pip install torch torchvision
    !pip install 'git+https://github.com/facebookresearch/pytorch3d.git'
    !pip install trimesh
    !pip install tensorflow-graphics

#2. Import libraries

In [ ]:
import os
import numpy as np
from tqdm.autonotebook import tqdm
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib as mpl

import trimesh
from tensorflow_graphics.notebooks import threejs_visualization

import torch
from pytorch3d.io import load_obj, save_obj
from pytorch3d.structures import Meshes
from pytorch3d.ops import sample_points_from_meshes, SubdivideMeshes
from pytorch3d.loss import (
    chamfer_distance, 
    mesh_edge_loss, 
    mesh_laplacian_smoothing, 
    mesh_normal_consistency,
)
print("Imports complete")

#3. Initialize GPU if available

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    cpu_device = torch.device("cpu")
    print("Device set to GPU")
else:
    device = torch.device("cpu")
    print("Device set to CPU")

#4. Conect to Google Drive and specify the paths

In [ ]:
path = "drive/My Drive/Colab Notebooks/Datasets/Models/"
input_path = path + 'input/datasets/'
target_path = path + 'target/datasets/'

#5. Load data

In [ ]:
def load_data(data_path):
    loaded_data = []
    print("Loading data")
    for obj in tqdm(sorted(os.listdir(data_path))):
        try:
            path = os.path.join(data_path, obj)
            verts, faces, aux = load_obj(path, load_textures=False)            
            loaded_data.append([np.array(verts), np.array(faces[0])])

        except Exception as e:
            print(str(e))

    return loaded_data

In [ ]:
def subdivide(data):
    meshes_subdivided = []
    print("Subdividing data")
    for i in tqdm(range(len(data))):
        mesh = Meshes(verts=[torch.from_numpy(data[i][0])], faces=[torch.from_numpy(data[i][1])])
        subdivided_mesh = SubdivideMeshes().forward(meshes=mesh)
        verts = subdivided_mesh.verts_list()
        faces = subdivided_mesh.faces_list()
        meshes_subdivided.append([np.array(verts[0]), np.array(faces[0])])

    return meshes_subdivided

In [ ]:
REBUILD_DATA = False
if REBUILD_DATA:
    input_models_test = load_data(input_path + "test/")
    input_models_train = load_data(input_path + "train/")

    target_models_test = load_data(target_path + "test/")
    target_models_train = load_data(target_path + "train/")

    input_models_test = subdivide(input_models_test)
    input_models_train = subdivide(input_models_train)

    np.save(path + 'input_models_train.npy', input_models_train, allow_pickle=True)
    np.save(path + 'input_models_test.npy', input_models_test, allow_pickle=True)
    np.save(path + 'target_models_train.npy', target_models_train, allow_pickle=True)
    np.save(path + 'target_models_test.npy', target_models_test, allow_pickle=True)

In [ ]:
LOAD_DATA = True
if LOAD_DATA:
    input_models_train = np.load(path + 'input_models_train.npy', allow_pickle=True)
    input_models_test = np.load(path + 'input_models_test.npy', allow_pickle=True)

    target_models_train = np.load(path + 'target_models_train.npy', allow_pickle=True)
    target_models_test = np.load(path + 'target_models_test.npy', allow_pickle=True)

In [ ]:
def validate_dataset(dataset):
    same_amount = True
    for i in range(np.array(dataset).shape[0]):
        if len(dataset[0][0]) != len(dataset[i][0]) or len(dataset[0][1]) != len(dataset[i][1]):
            same_amount = False
    if same_amount:
        print("Vertices: ", len(dataset[0][0]))
        print("Faces: ", len(dataset[0][1]))
    else:
        print("Meshes has different amount of vertices and faces")

In [ ]:
validate_dataset(input_models_test)
validate_dataset(input_models_train)
validate_dataset(target_models_train)
validate_dataset(target_models_test)

#6. Visualize some meshes to ensure the data was loaded correctly

In [ ]:
display_size = 500
id_train = 0
id_test = 0

input_test_mesh = {"vertices": input_models_test[id_test][0], "faces": input_models_test[id_test][1]}
input_train_mesh = {"vertices": input_models_train[id_train][0], "faces": input_models_train[id_train][1]}
target_test_mesh = {"vertices": target_models_test[id_test][0], "faces": target_models_test[id_test][1]}
target_train_mesh = {"vertices": target_models_train[id_train][0], "faces": target_models_train[id_train][1]}

threejs_visualization.triangular_mesh_renderer(input_train_mesh, width=display_size, height=display_size)
threejs_visualization.triangular_mesh_renderer(target_train_mesh, width=display_size, height=display_size)
threejs_visualization.triangular_mesh_renderer(input_test_mesh, width=display_size, height=display_size)
threejs_visualization.triangular_mesh_renderer(target_test_mesh, width=display_size, height=display_size)

#PART 2 - Deep Learning
###2.1 Build network

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from google.colab import output

torch.set_default_tensor_type('torch.cuda.FloatTensor')

In [ ]:
neurons = 5000
dropout_rate = 0.5

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fci = nn.Linear(202 * 3 + 400 * 3, neurons)
        self.fch1 = nn.Linear(neurons, neurons)
        self.fch2 = nn.Linear(neurons, neurons)
        self.fch3 = nn.Linear(neurons, neurons)
        self.fch4 = nn.Linear(neurons, neurons)
        self.fch5 = nn.Linear(neurons, neurons)
        self.fch6 = nn.Linear(neurons, neurons)
        self.fch7 = nn.Linear(neurons, neurons)
        self.fco = nn.Linear(neurons, 202 * 3)

        self.dropout1 = nn.Dropout(p=dropout_rate)
        self.dropout2 = nn.Dropout(p=dropout_rate)
        self.dropout3 = nn.Dropout(p=dropout_rate)
        self.dropout4 = nn.Dropout(p=dropout_rate)
        self.dropout5 = nn.Dropout(p=dropout_rate)
        self.dropout6 = nn.Dropout(p=dropout_rate)
        self.dropout7 = nn.Dropout(p=dropout_rate)

    def forward(self, x):
        x = F.relu(self.fci(x))
        x = self.fch1(x)
        x = self.dropout1(x)
        x = F.relu(x)
        x = self.fch2(x)
        x = self.dropout2(x)
        x = F.relu(x)
        x = self.fch3(x)
        x = self.dropout3(x)
        x = F.relu(x)
        x = self.fch4(x)
        x = self.dropout4(x)
        x = F.relu(x)
        x = self.fch5(x)
        x = self.dropout5(x)
        x = F.relu(x)
        x = self.fch6(x)
        x = self.dropout6(x)
        x = F.relu(x)
        x = self.fch7(x)
        x = self.dropout7(x)
        x = F.relu(x)
        x = self.fco(x)
        return x

net = Net()
net.to(device)
print(net)

In [ ]:
optimizer = optim.Adam(net.parameters(), lr=0.000005)

w_chamfer = 1.0
w_chamfer_normals = 1.0
w_edge = 1.0
w_normal = 0.01
w_laplacian = 0.1

epochs = 100
dataset_size = len(input_models_train)

print("Optimizer: " + str(optimizer))
print("Loss function weights" + "\nChamfer distance: " + str(w_chamfer) + "\nChamfer normals distance: " + str(w_chamfer_normals) + "\nEdge length weight: " + str(w_edge) + "\nNormal consistency: " + str(w_chamfer) +  "\nLaplacian smoothing: " + str(w_laplacian))
print("Epochs: " + str(epochs))
print("Dataset size: " + str(dataset_size))

In [ ]:
total_losses = []
chamfer_losses = []
chamfer_normals_losses = []
edge_losses = []
normal_losses = []
laplacian_losses = []

In [ ]:
import time
start_time = time.time()

for epoch in tqdm(range(epochs)):
    for i in range(dataset_size):
        net.zero_grad()

        input_mesh_verts = torch.from_numpy(input_models_train[i][0]).cuda()
        input_mesh_faces = torch.from_numpy(input_models_train[i][1]).cuda()

        target_mesh_verts = torch.from_numpy(target_models_train[i][0]).cuda()
        target_mesh_faces = torch.from_numpy(target_models_train[i][1]).cuda()

        net_input = torch.flatten(torch.cat((input_mesh_verts, torch.tensor(input_mesh_faces, dtype=torch.float32)), dim=0)).cuda()
        net_output = net(net_input)

        output_mesh_verts = input_mesh_verts + net_output.view(-1, 3)
        output_mesh_faces = input_mesh_faces

        output_mesh = Meshes(verts=[output_mesh_verts], faces=[output_mesh_faces])
        target_mesh = Meshes(verts=[target_mesh_verts], faces=[target_mesh_faces])

        sample_output_mesh, output_normals = sample_points_from_meshes(output_mesh, 5000, return_normals=True)
        sample_target_mesh, target_normals = sample_points_from_meshes(target_mesh, 5000, return_normals=True)

        loss_chamfer, loss_chamfer_normals = chamfer_distance(sample_output_mesh, sample_target_mesh, x_normals=output_normals, y_normals=target_normals)
        loss_edge = mesh_edge_loss(output_mesh)
        loss_normal = mesh_normal_consistency(output_mesh)
        loss_laplacian = mesh_laplacian_smoothing(output_mesh, method="uniform")



        loss = loss_chamfer * w_chamfer + loss_chamfer_normals * w_chamfer_normals + loss_edge * w_edge + loss_normal * w_normal + loss_laplacian * w_laplacian

        loss.backward()
        optimizer.step()

        total_losses.append(loss.item())
        chamfer_losses.append(loss_chamfer.item())
        chamfer_normals_losses.append(loss_chamfer_normals.item())
        edge_losses.append(loss_edge.item())
        normal_losses.append(loss_normal.item())
        laplacian_losses.append(loss_edge.item())
        
print("Execution time %s seconds" % round((time.time() - start_time), 2))

#2.2 Plot losses from training

In [ ]:
import matplotlib.pyplot as plt
def plot_losses(input_losses, title, i_dataset_size, i_epochs):
    plt.plot(input_losses) 
    plt.title(title)
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.show()

In [ ]:
plot_losses(total_losses, 'Total loss', dataset_size, epochs)
plot_losses(chamfer_losses, 'Chamfer distance loss', dataset_size, epochs)
plot_losses(chamfer_normals_losses, 'Chamfer distance normals loss', dataset_size, epochs)
plot_losses(edge_losses, 'Mesh edge loss', dataset_size, epochs)
plot_losses(normal_losses, 'Mesh normal consistency loss', dataset_size, epochs)
plot_losses(laplacian_losses, 'Mesh laplacian smoothing loss', dataset_size, epochs)

#Plot average losses from training

In [ ]:
import matplotlib.pyplot as plt
def average_loss(input_losses, dataset_size, epochs):
    average_losses = []
    for e in range(epochs):
        epoch_losses = []
        for i in range(dataset_size):
            loss = input_losses[(e*dataset_size)+i]
            epoch_losses.append(loss)
        epoch_average = np.average(epoch_losses)
        average_losses.append(epoch_average)
    return average_losses

In [ ]:
average_total_losses =  average_loss(total_losses, dataset_size, epochs)
average_chamfer_losses =  average_loss(chamfer_losses, dataset_size, epochs)
average_chamfer_normals_losses =  average_loss(chamfer_normals_losses, dataset_size, epochs)
average_edge_losses =  average_loss(edge_losses, dataset_size, epochs)
average_normal_losses =  average_loss(normal_losses, dataset_size, epochs)
average_laplacian_losses =  average_loss(laplacian_losses, dataset_size, epochs)

In [ ]:
plt.plot(average_total_losses, linewidth=3,  label="Total")
plt.plot(average_chamfer_losses, linestyle='dashed', label="Chamfer")
plt.plot(average_chamfer_normals_losses, linestyle='dashed', label="Chamfer Normals")
plt.plot(average_edge_losses, linestyle='dashed', label="Edges")
plt.plot(average_normal_losses, linestyle='dashed', label="Normals")
plt.plot(average_laplacian_losses, linestyle='dashed', label="Laplacian")
plt.title("Average of the different losses per epoch")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
average_total_losses

#Training total loss with moving average

In [ ]:
import matplotlib.pyplot as plt
def plot_losses_mvavg(input_losses, title, epochs):
    excact_losses = []
    for i in range(i_dataset_size * i_epochs):
        loss = input_losses[i].item()
        extracted_losses.append(loss)
    
    plt.plot(extracted_losses) 
    plt.title(title)
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.show()

# Test on training- and testset

In [ ]:
def test_model(input_models, target_models, dataset_size):

    total_losses = []
    chamfer_losses = []
    chamfer_normals_losses = []
    edge_losses = []
    normal_losses = []
    laplacian_losses = []

    with torch.no_grad():
        for i in tqdm(range(dataset_size)):
            input_mesh_verts = torch.from_numpy(input_models[i][0]).cuda()
            input_mesh_faces = torch.from_numpy(input_models[i][1]).cuda()

            target_mesh_verts = torch.from_numpy(target_models[i][0]).cuda()
            target_mesh_faces = torch.from_numpy(target_models[i][1]).cuda()

            net_input = torch.flatten(torch.cat((input_mesh_verts, torch.tensor(input_mesh_faces, dtype=torch.float32)), dim=0)).cuda()
            net_output = net(net_input)

            output_mesh_verts = input_mesh_verts + net_output.view(-1, 3)
            output_mesh_faces = input_mesh_faces

            output_mesh = Meshes(verts=[output_mesh_verts], faces=[output_mesh_faces])
            target_mesh = Meshes(verts=[target_mesh_verts], faces=[target_mesh_faces])

            sample_output_mesh, output_normals = sample_points_from_meshes(output_mesh, 5000, return_normals=True)
            sample_target_mesh, target_normals = sample_points_from_meshes(target_mesh, 5000, return_normals=True)

            loss_chamfer, loss_chamfer_normals = chamfer_distance(sample_output_mesh, sample_target_mesh, x_normals=output_normals, y_normals=target_normals)
            loss_edge = mesh_edge_loss(output_mesh)
            loss_normal = mesh_normal_consistency(output_mesh)
            loss_laplacian = mesh_laplacian_smoothing(output_mesh, method="uniform")

            chamfer_losses.append(loss_chamfer)
            chamfer_normals_losses.append(loss_chamfer_normals)
            edge_losses.append(loss_edge)
            normal_losses.append(loss_normal)
            laplacian_losses.append(loss_edge)

            loss = loss_chamfer * w_chamfer + loss_chamfer_normals * w_chamfer_normals + loss_edge * w_edge + loss_normal * w_normal + loss_laplacian * w_laplacian

            total_losses.append(loss)
    
    return total_losses, chamfer_losses, chamfer_normals_losses, edge_losses, normal_losses, laplacian_losses     

In [ ]:
 train_total_losses, train_chamfer_losses, train_chamfer_normals_losses, train_edge_losses, train_normal_losses, train_laplacian_losses = test_model(input_models_train, target_models_train, 500)

In [ ]:
 test_total_losses, test_chamfer_losses, test_chamfer_normals_losses, test_edge_losses, test_normal_losses, test_laplacian_losses = test_model(input_models_test, target_models_test, 500)

In [ ]:
def print_average(input_losses):
    losses = []
    for i in range(len(input_losses)):
        loss = input_losses[i].item()
        losses.append(loss)
    return round(np.mean(losses), 3)

In [ ]:
print("Training set average total loss")
print_average(train_total_losses)

In [ ]:
print("Test set average total loss")
print_average(test_total_losses)

# Visualize 3D models

In [ ]:
def visualize_results(input_set, target_set, mesh_id):
    display_size = 400
    camera = threejs_visualization.build_perspective_camera(enable_zoom=True)

    with torch.no_grad():
        input_mesh_verts = torch.from_numpy(input_set[mesh_id][0]).cuda()
        input_mesh_faces = torch.from_numpy(input_set[mesh_id][1]).cuda()

        target_mesh_verts = torch.from_numpy(target_set[mesh_id][0]).cuda()
        target_mesh_faces = torch.from_numpy(target_set[mesh_id][1]).cuda()

        net_input = torch.flatten(torch.cat((input_mesh_verts, torch.tensor(input_mesh_faces, dtype=torch.float32)), dim=0)).cuda()
        net_output = net(net_input)

        output_mesh_verts = input_mesh_verts + net_output.view(-1, 3)
        output_mesh_faces = input_mesh_faces

        input_mesh = Meshes(verts=[input_mesh_verts], faces=[input_mesh_faces])
        output_mesh = Meshes(verts=[output_mesh_verts], faces=[output_mesh_faces])
        target_mesh = Meshes(verts=[target_mesh_verts], faces=[target_mesh_faces])

        input_mesh_trimesh = {"vertices": np.array(input_mesh.verts_list()[0].tolist()), "faces": np.array(input_mesh.faces_list()[0].tolist())}
        output_mesh_trimesh = {"vertices": np.array(output_mesh.verts_list()[0].tolist()), "faces": np.array(output_mesh.faces_list()[0].tolist())}
        target_mesh_triemsh = {"vertices": np.array(target_mesh.verts_list()[0].tolist()), "faces": np.array(target_mesh.faces_list()[0].tolist())}

        threejs_visualization.triangular_mesh_renderer(input_mesh_trimesh, width=display_size, height=display_size, camera=camera)
        threejs_visualization.triangular_mesh_renderer(output_mesh_trimesh, width=display_size, height=display_size, camera=camera)
        threejs_visualization.triangular_mesh_renderer(target_mesh_triemsh, width=display_size, height=display_size, camera=camera)

In [ ]:
#Train
visualize_results(input_models_train, target_models_train, 0)

In [ ]:
#Test
visualize_results(input_models_test, target_models_test, 0)